In [ ]:
using LazySets, Plots
using LazySets: project, Interval

In [ ]:
X = Interval(1.0, 2)

In [ ]:
f(x) = 2*x;

In [ ]:
F(X) = Interval(f(X.dat));  # f(X)

In [ ]:
Y = cartesian_product(X, F(X))  # X × f(X)

In [ ]:
Z = LineSegment(low(Y), high(Y))  # {(x, f(x)) | x ∈ X}

In [ ]:
plot(leg=:outerright)
plot!(Y, lab="X × f(X)")
plot!(Z, lab="{(x, f(x)) | x ∈ X}")

In [ ]:
using ReachabilityAnalysis, Polyhedra, DifferentialEquations
const DE = DifferentialEquations;

In [ ]:
A = [0.0 1; 0 0]

In [ ]:
@taylorize function dyn_nln!(dx, x, p, t)
    dx[1] = x[2]
    dx[2] = zero(x[2])
    return nothing
end;

In [ ]:
U = ZeroSet(1)
XU = cartesian_product(X, U)
#XU = LineSegment([1.0, 2], [2.0, 4])
ivp_lin = @ivp(x' = A * x, x(0) ∈ XU)
ivp_nln = @ivp(x' = dyn_nln!(x), dim: 2, x(0) ∈ XU)
period = 0.1
T = 0.2;
tspan = 0.0 .. T
cpost_lin = ReachabilityAnalysis._default_cpost(ivp_lin, tspan)
@show cpost_lin2 = GLGM06(δ=1e-2, max_order=10, approx_model=FirstOrderZonotope())
cpost_nln = ReachabilityAnalysis._default_cpost(ivp_nln, tspan)
cpost_nln2 = TMJets21b(orderQ=4, orderT=3)
steps = ceil(Int, T / period)
lab(v) = v == 0 ? "t" : v == 1 ? "x" : "u";

In [ ]:
controller = X -> f(X);

In [ ]:
function _solve_ensemble(ivp, X0_samples, tspan;
                         trajectories_alg=DE.Tsit5(),
                         ensemble_alg=DE.EnsembleThreads(),
                         inplace=true,
                         kwargs...)
    if inplace
        field = ReachabilityAnalysis.inplace_field!(ivp)
    else
        field = ReachabilityAnalysis.outofplace_field(ivp)
    end

    _prob_func(prob, i, repeat) = remake(prob, u0=X0_samples[i])
    ensemble_prob = EnsembleProblem(ODEProblem(field, first(X0_samples), tspan),
                                    prob_func=_prob_func)
    return DE.solve(ensemble_prob, trajectories_alg, ensemble_alg;
                    trajectories=length(X0_samples))
end;

In [ ]:
function simulate(ivp, controller, steps; trajectories=1, include_vertices=true)
    X0 = project(initial_state(ivp), [1])
    t1 = 0.0
    
    states = sample(X0, trajectories; include_vertices=include_vertices)
    # println(states)
    trajectories = length(states)
    extended = Vector{Vector{Float64}}(undef, trajectories)
    simulations = Vector{EnsembleSolution}(undef, steps)
    
    for k in 1:steps
        # println("k = $k")
        for j in 1:trajectories
            x0 = states[j]
            control = controller(x0)
            extended[j] = vcat(x0, control)
            # println(extended[j])
        end
        
        t2 = t1 + period
        simulations[k] = _solve_ensemble(ivp, extended, (t1, t2); inplace=true)
        
        for j in 1:trajectories
            ode_solution = simulations[k][j]
            final_extended = ode_solution.u[end]
            states[j] = final_extended[1:1]
        end
        t1 = t2
    end
    return simulations
end;

In [ ]:
function solve1(ivp, controller, steps, cpost; time_shift=false)
    S = system(ivp)
    XU = initial_state(ivp)
    t = 0.0
    
    NT = ReachabilityAnalysis.numtype(cpost)
    RT = rsetrep(cpost)
    FT = Flowpipe{NT, RT, Vector{RT}}
    out = Vector{FT}(undef, steps)
    X0s = Vector{Hyperrectangle}(undef, steps)
    solts = Vector(undef, steps)
    
    for k in 1:steps     
        X = convert(Interval, project(XU, [1]))
        U = convert(Interval, controller(X))  # next inputs
        
        X0_cont = cartesian_product(X, U)
        X0s[k] = X0_cont
        
        if time_shift
            dt = 0 .. period
        else
            dt = t .. t + period
        end
        sol = ReachabilityAnalysis.post(cpost, IVP(S, X0_cont), dt)
        if time_shift
            sol = shift(sol, t)
        end
        out[k] = sol
        
        t = t + period
        XU = sol(t)
        if XU isa TaylorModelReachSet
            XU = evaluate(XU, t)
        end
        XU = set(overapproximate(XU, Zonotope))
        solts[k] = XU
    end
    return MixedFlowpipe(out), X0s, solts
end;

In [ ]:
function solve2(ivp, controller, steps, cpost; time_shift=false)
    S = system(ivp)
    XU = initial_state(ivp)
    t = 0.0
    
    NT = ReachabilityAnalysis.numtype(cpost)
    RT = rsetrep(cpost)
    FT = Flowpipe{NT, RT, Vector{RT}}
    out = Vector{FT}(undef, steps)
    X0s = Vector{LineSegment}(undef, steps)
    solts = Vector(undef, steps)
    
    for k in 1:steps     
        X = convert(Interval, project(XU, [1]))
        Y = cartesian_product(X, F(X))  # X × f(X)
        
        X0_cont = LineSegment(low(Y), high(Y))  # {(x, f(x)) | x ∈ X}
        println("X0_cont = $X0_cont")
        X0s[k] = X0_cont
        
        if time_shift
            dt = 0 .. period
        else
            dt = t .. t + period
        end
        sol = ReachabilityAnalysis.post(cpost, IVP(S, X0_cont), dt)
        if time_shift
            sol = shift(sol, t)
        end
        out[k] = sol
        
        t = t + period
        XU = sol(t)
        println("XU1 = $XU")
        if XU isa TaylorModelReachSet
            XU = evaluate(XU, t)
            println("XU2 = $XU")
        end
        XU = set(overapproximate(XU, Zonotope))
        println("XU3 = $XU")
        solts[k] = XU
    end
    return MixedFlowpipe(out), X0s, solts
end;

In [ ]:
sim = simulate(ivp_nln, controller, steps, trajectories=100)
S1, X0s1, solts1 = solve1(ivp_nln, controller, steps, cpost_nln2)
S2, X0s2, solts2 = solve2(ivp_nln, controller, steps, cpost_nln2);

In [ ]:
vars = (1, 2)
fig = plot(xlab=lab(vars[1]), ylab=lab(vars[2]), leg=:outerright)
plot!(fig, S1, vars=vars, lab="S1")
plot!(Y)
plot!(fig, S2, vars=vars, lab="S2")
#plot!(fig, overapproximate(S1[1][end], Hyperrectangle, nsdiv=10), vars=vars, lab="S1-1")
#plot!(fig, overapproximate(S2[1][end], Hyperrectangle, nsdiv=10), vars=vars, lab="S2-1")
#plot!(fig, S1[2], vars=vars, lab="S1-2")
#plot!(fig, S2[2], vars=vars, lab="S2-2")
if 0 ∉ vars
    plot!(fig, LineSegment([low(X, 1), low(F(X), 1)], [high(X, 1), high(F(X), 1)]), c=:yellow, lab="X0")
    plot!(fig, LineSegment([1.2, 2.4], [2.4, 4.8]), c=:orange, lab="X0-2")
end
#plot!(X0s1[1], lab="X01-1")
#plot!(X0s2[1], lab="X02-1")
#plot!(X0s1[2], lab="X01-2")
#plot!(X0s2[2], lab="X02-2")
xl = Plots.xlims(fig)
yl = Plots.ylims(fig)
c = :red
for simulation in sim
    for piece in simulation
        Plots.plot!(fig, piece, vars=vars, color=c, lab="")
    end
    c = :orange
    #break  # only show first step
end
Plots.xlims!(fig, xl)
Plots.ylims!(fig, yl)
fig

In [ ]:
S2_set = S2[1].Xk

In [ ]:
TM = evaluate.(S2_set, S2_set[1].Δt)[1]

In [ ]:
Z1 = overapproximate(TM, Zonotope)

In [ ]:
convert(Zonotope, Z)

In [ ]:
TM2 = overapproximate(Z, TaylorModelReachSet)

In [ ]:
overapproximate(TM2, Zonotope)

In [ ]:
sim = simulate(ivp_lin, controller, steps, trajectories=100)
S1, X0s1, solts1 = solve1(ivp_lin, controller, steps, cpost_lin2; time_shift=true)
S2, X0s2, solts2 = solve2(ivp_lin, controller, steps, cpost_lin2; time_shift=true);

In [ ]:
vars = (1, 2)
fig = plot(xlab=lab(vars[1]), ylab=lab(vars[2]), leg=:outerright)
plot!(fig, S1, vars=vars, lab="S1")
plot!(Y)
plot!(fig, S2, vars=vars, lab="S2")
#plot!(fig, S1[2], vars=vars, lab="S1-2")
#plot!(fig, S2[2], vars=vars, lab="S2-2")
if 0 ∉ vars
    plot!(fig, LineSegment([low(X, 1), low(F(X), 1)], [high(X, 1), high(F(X), 1)]), c=:yellow, lab="X0")
    plot!(fig, LineSegment([1.2, 2.4], [2.4, 4.8]), c=:orange, lab="X0-2")
end
#plot!(X0s1[1], lab="X01-1")
#plot!(X0s2[1], lab="X02-1")
#plot!(X0s1[2], lab="X01-2")
#plot!(X0s2[2], lab="X02-2")
xl = Plots.xlims(fig)
yl = Plots.ylims(fig)
c = :red
for simulation in sim
    for piece in simulation
        Plots.plot!(fig, piece, vars=vars, color=c, lab="")
    end
    c = :orange
    #break  # only show first step
end
Plots.xlims!(fig, xl)
Plots.ylims!(fig, yl)
fig